In [6]:
from __future__ import division

from sympy.interactive.printing import init_printing
init_printing(use_unicode=False, wrap_line=True, use_latex='mathjax')
# from sympy.interactive import init_session
# init_session() 
from sympy.matrices import Matrix, eye, zeros, diag
from sympy import symbols, diff
import sympy as sp
import numpy as np

#consider a 2 robot system.
A1 = zeros(12, 12)
B1 = zeros(12, 4)
gravity = sp.Symbol('g')
A1[6, 1] = gravity
A1[7, 0] = -gravity
A1[0:3, 3:6] = eye(3)
A1[9:, 6:9] = eye(3)
A1

[0   0  0  1  0  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  1  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  1  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[0   g  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[-g  0  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  1  0  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  1  0  0  0  0]
[                                   ]
[0   0  0  0  0  0  0  0  1  0  0  0]

In [7]:
mass = symbols('m')
Ixx, Iyy, Izz = symbols('Ixx Iyy Izz')
B1[8, 0] = 1 / mass
B1[3:6, 1:] = diag([1 / Ixx, 1 / Iyy, 1 / Izz], unpack=True)
B1

[0   0    0    0 ]
[                ]
[0   0    0    0 ]
[                ]
[0   0    0    0 ]
[                ]
[    1           ]
[0  ---   0    0 ]
[   Ixx          ]
[                ]
[         1      ]
[0   0   ---   0 ]
[        Iyy     ]
[                ]
[              1 ]
[0   0    0   ---]
[             Izz]
[                ]
[0   0    0    0 ]
[                ]
[0   0    0    0 ]
[                ]
[1               ]
[-   0    0    0 ]
[m               ]
[                ]
[0   0    0    0 ]
[                ]
[0   0    0    0 ]
[                ]
[0   0    0    0 ]

In [8]:
n = 2
A2 = A1.copy()
B2 = B1.copy()
A = diag([A1, A2], unpack=True)
B = diag([B1, B2], unpack=True)
x1 = Matrix(symbols('x:12'))
u1 = Matrix(symbols('u:4'))
x2 = Matrix(symbols('x12:24'))
u2 = Matrix(symbols('u4:8'))
x = Matrix.vstack(x1, x2)
u = Matrix.vstack(u1, u2)
c = sp.symbols('c')
Ds = Matrix([sp.symbols('Ds')])
# Ds = 1
print(x.shape)
print(u.shape)


(24, 1)
(8, 1)


In [9]:
e10 = Matrix([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
e11 = Matrix([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
e12 = Matrix([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Exy = Matrix([e10.T, e11.T])
Exy

[0  0  0  0  0  0  0  0  0  1  0  0]
[                                  ]
[0  0  0  0  0  0  0  0  0  0  1  0]

In [10]:
Ez = e12.T
Ez

[0  0  0  0  0  0  0  0  0  0  0  1]

In [11]:
def norm2(x):
    return sp.sqrt(x.T @ x)

In [12]:
def h(xi, xj):
    return norm2(Exy @ (xi - xj)) ** 4 + ((Ez / c) @ (xi - xj)) ** 4 - Ds ** 4


In [13]:
# tmp = (Exy@(x1-x2)).T @ (Exy@(x1-x2))
# tmp2 = tmp**4
h12 = h(x1, x2)[0]
h12
# tmp1 = norm2(Exy@(x1-x2))**4 
# tmp2 = ((Ez / c) @ (x1 - x2))**4

                                     2              4
    4   /           2              2\    (x11 - x23) 
- Ds  + \(x10 - x22)  + (-x21 + x9) /  + ------------
                                               4     
                                              c      

In [14]:
dhdx = h12.diff(x)
dhdx = dhdx.reshape(24, 1)
dhdx = Matrix(dhdx)
dhdx


[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                      0                       ]
[                                              ]
[                /           2              2\ ]
[(-4*x21 + 4*x9)*\(x10 - x22)  + (-x21 + x9) / ]
[                   

In [15]:

#compare this dhdx to my hand calculation

def hand_dhdx(xi, xj, i=0, j=1):
    #this is the gradient wrt the large x of (xi - xj)
    grad_x_sub = zeros(12 * n, 12)
    grad_x_sub[12 * i:12 * (i + 1), :] = eye(12)
    grad_x_sub[12 * j:12 * (j + 1), :] = -eye(12)
    #this calculation looks correct

    return 4 * (norm2(Exy @ (xi - xj)) ** 2)[0] * grad_x_sub @ (Exy.T @ Exy) @ (xi - xj) + (4 / c) * \
        (((Ez / c) @ (xi - xj)) ** 3)[0] * grad_x_sub @ Ez.T


my_dhdx = hand_dhdx(x1, x2)
my_dhdx

[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[                       0                       ]
[                                               ]
[             /             2                2\ ]
[ (-x21 + x9)*\4*(x10 - x22)  + 4*(-x21 + x9) / ]


In [16]:
#compare the two
difference = my_dhdx - dhdx
difference.simplify()
# we can see that the two are the same
difference

[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]
[ ]
[0]

In [17]:
#now lets check that the time derivative of h doesnt have a component in
(my_dhdx.T @ B)
#clearly all zeros, so we look at the next derivative


[0  0  0  0  0  0  0  0]

In [18]:
d2hdx2 = diff(dhdx, x)
d2hdx2 = d2hdx2.reshape(24, 24)
# print(d2hdx2) #this is the second derivative of h wrt x
# d2hdx2[8:, 8:13]

In [19]:
#compare to my hand calculation
def hand_d2hdx2(xi, xj, i=0, j=1):
    grad_x_sub = zeros(12 * n, 12)
    grad_x_sub[12 * i:12 * (i + 1), :] = eye(12)
    grad_x_sub[12 * j:12 * (j + 1), :] = -eye(12)
    e_12iz = zeros(12 * n, 1)
    e_12iz[12 * i + 11] = 1
    e_12jz = zeros(12 * n, 1)
    e_12jz[12 * j + 11] = 1
    #ive checked the second term and it seems fine
    return grad_x_sub @ (8 * Exy.T @ Exy @ (xi - xj) @ ((Exy @ (xi - xj)).T @ Exy) + 4 * (norm2(Exy @ (xi - xj)) ** 2)[
        0] * Exy.T @ Exy) @ grad_x_sub.T + (12 / c ** 2) * (((Ez / c) @ (xi - xj)) ** 2)[0] * (e_12iz - e_12jz) @ (
                e_12iz - e_12jz).T

# xi = x1
# xj = x2
# i=0
# j=1
# grad_x_sub = zeros(12*n, 12)
# grad_x_sub[12*i:12*(i+1), :] = eye(12)
# grad_x_sub[12*j:12*(j+1), :] = -eye(12) 
# e_12iz = zeros(12*n, 1)
# e_12iz[12*i + 11] = 1
# e_12jz = zeros(12*n, 1)
# e_12jz[12*j + 11] = 1
# pt1 = grad_x_sub @ (8*Exy.T @ Exy @ (xi-xj) @ ((Exy @ (xi-xj)).T @ Exy) + 4* (norm2(Exy @ (xi-xj))**2)[0] * Exy.T @ Exy) @ grad_x_sub.T
# 
# pt2 = (12/c**2) * (((Ez / c) @ (xi - xj))**2)[0] * (e_12iz - e_12jz) @ (e_12iz - e_12jz).T
# pt2

In [29]:
my_d2hdx2 = hand_d2hdx2(x1, x2)
# my_d2hdx2[8:, 8:13]
my_d2hdx2

[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                                 0                  
[                                                                             
[0  0  0  0  0  0  0  0  0                          

In [21]:
#compare
difference2 = sp.ImmutableDenseNDimArray(my_d2hdx2) - d2hdx2
difference2 = difference2.simplify()
found_nonzero = False
for i in range(24):
    for j in range(24):
        if difference2[i, j] != 0:
            found_nonzero = True
            print(i, j, difference2[i, j])

if not found_nonzero:
    print('All zeros')
else:
    print('Not all zeros')

All zeros


In [22]:
from scipy.spatial.transform import Rotation


def rpy_to_rot(rpy):
    roll, pitch, yaw = rpy
    sin = sp.sin
    cos = sp.cos
    
    yawMatrix = Matrix([
        [cos(yaw), -sin(yaw), 0],
        [sin(yaw), cos(yaw), 0],
        [0, 0, 1]
    ])


    pitchMatrix = Matrix([
        [cos(pitch), 0, sin(pitch)],
        [0, 1, 0],
        [-sin(pitch), 0, cos(pitch)]
    ])
    
    rollMatrix = Matrix([
        [1, 0, 0],
        [0, cos(roll), -sin(roll)],
        [0, sin(roll), cos(roll)]
    ])
    return yawMatrix @ pitchMatrix @ rollMatrix

In [23]:

#symbolic xdot
xref = Matrix(symbols('x^{*}_{:24}'))
u_eq = Matrix([mass * gravity, 0, 0, 0, mass*gravity, 0, 0, 0])
#essentially we always have

R_ref = eye(3)
R_ref[0, 0] = sp.cos(xref[2])
R_ref[0, 1] = -sp.sin(xref[2])
R_ref[1, 0] = sp.sin(xref[2])
R_ref[1, 1] = sp.cos(xref[2])
R = rpy_to_rot(x[:3])
#think about how to do this properly? for now if we assume the desired yaw is 0, then we have purely 
xhat = x - xref
uhat = u - u_eq

# R = Rotation.from_euler('xyz', (1,2,3)).as_matrix()

# e = x - xref
# e
# this is how we form the error state using the desired yaw. essential R_des.T @ (x-xdes) where R_des is the desired yaw angle
# R_eq = Rotation.from_euler('xyz', [0, 0, x_des[2]]).as_matrix()
# R = Rotation.from_euler('xyz', x[:3]).as_matrix()
# R_err = R_eq.T @ R
# e[:3] = Rotation.from_matrix(R_err).as_euler('xyz')
# e[9:] = R_eq.T @ (x[9:] - x_des[9:])
# e[6:9] = R_eq.T @ (x[6:9] - x_des[6:9])
# e[3:6] = R_eq.T @ (x[3:6] - x_des[3:6])

xdot = A @ xhat + B @ uhat
xdot
# xdot
#need to think about how thi

[   x3 - x^{*}_{3}    ]
[                     ]
[   x4 - x^{*}_{4}    ]
[                     ]
[   x5 - x^{*}_{5}    ]
[                     ]
[          u1         ]
[         ---         ]
[         Ixx         ]
[                     ]
[          u2         ]
[         ---         ]
[         Iyy         ]
[                     ]
[          u3         ]
[         ---         ]
[         Izz         ]
[                     ]
[ g*(x1 - x^{*}_{1})  ]
[                     ]
[ -g*(x0 - x^{*}_{0}) ]
[                     ]
[      -g*m + u0      ]
[      ---------      ]
[          m          ]
[                     ]
[   x6 - x^{*}_{6}    ]
[                     ]
[   x7 - x^{*}_{7}    ]
[                     ]
[   x8 - x^{*}_{8}    ]
[                     ]
[  x15 - x^{*}_{15}   ]
[                     ]
[  x16 - x^{*}_{16}   ]
[                     ]
[  x17 - x^{*}_{17}   ]
[                     ]
[          u5         ]
[         ---         ]
[         Ixx         ]
[               

In [24]:
hdot = my_dhdx.T @ xdot
hdot


[                                                                             
[                               /               2                2\           
[(x10 - x22)*(x19 - x^{*}_{19})*\- 4*(x10 - x22)  - 4*(-x21 + x9) / + (x10 - x
[                                                                             
[                                                                             

                                                                              
                     /             2                2\                        
22)*(x7 - x^{*}_{7})*\4*(x10 - x22)  + 4*(-x21 + x9) / + (x18 - x^{*}_{18})*(-
                                                                              
                                                                              

                                                                              
          /               2                2\                                /
x21 + x9)*\- 4*(x10 - x22)  - 4*(-x21 + x9) / + (-

In [25]:
x_2dot = A @ A @ xhat + A @ B @ uhat
x_2dot

[          u1         ]
[         ---         ]
[         Ixx         ]
[                     ]
[          u2         ]
[         ---         ]
[         Iyy         ]
[                     ]
[          u3         ]
[         ---         ]
[         Izz         ]
[                     ]
[          0          ]
[                     ]
[          0          ]
[                     ]
[          0          ]
[                     ]
[ g*(x4 - x^{*}_{4})  ]
[                     ]
[ -g*(x3 - x^{*}_{3}) ]
[                     ]
[          0          ]
[                     ]
[ g*(x1 - x^{*}_{1})  ]
[                     ]
[ -g*(x0 - x^{*}_{0}) ]
[                     ]
[      -g*m + u0      ]
[      ---------      ]
[          m          ]
[                     ]
[          u5         ]
[         ---         ]
[         Ixx         ]
[                     ]
[          u6         ]
[         ---         ]
[         Iyy         ]
[                     ]
[          u7         ]
[         ---   

In [26]:
h_2dot = my_dhdx.T @ x_2dot + (A @ xhat).T @ (my_d2hdx2.T @ (A @ xhat + B @ uhat)) 
h_2dot.simplify()

# easier_h2dot = tmp @ (A@xhat + B@uhat)


In [27]:
term1 = my_dhdx.T @ A@B@uhat #as I expected this only considers the thrust inputs, to encode the full dynamics we need to consider the full input, and therefore another derivative
term1 

[             3                            3            ]
[4*(x11 - x23) *(-g*m + u0)   4*(x11 - x23) *(-g*m + u4)]
[-------------------------- - --------------------------]
[            4                            4             ]
[           c *m                         c *m           ]

In [28]:
term2 = (A @ xhat).T @ (my_d2hdx2.T @ (A@xhat + B @ uhat))
term2

[                                                                             
[                   /                                                         
[(x18 - x^{*}_{18})*\(8*x10 - 8*x22)*(x19 - x^{*}_{19})*(-x21 + x9) - (8*x10 -
[                                                                             
[                                                                             

                                                                              
                                                          /             2     
 8*x22)*(-x21 + x9)*(x7 - x^{*}_{7}) + (x18 - x^{*}_{18})*\4*(x10 - x22)  + (-
                                                                              
                                                                              

                                                                              
                                         2\                    /              
8*x21 + 8*x9)*(-x21 + x9) + 4*(-x21 + x9) / + (x6 

In [ ]:
x_3dot = (A@A) @ (A @ xhat + B @ uhat)
x_3dot


# thrust (Vdot) -> vz -> z
# omega -> rpy proportional to ax/ay -> vx/vy -> x,y 
# thrust + omega control in sim
    

# d_h2dotdx = diff(h_2dot, x)
# d_h2dotdx = d_h2dotdx.reshape(24,1)
# d_h2dotdx = Matrix(d_h2dotdx)
# print(d_h2dotdx.shape)
# print(type(d_h2dotdx))
# print(sp.latex(d_h2dotdx))

In [ ]:
h_3dot = d_h2dotdx.T @ (A@xhat + B@uhat)
h_3dot.simplify()
print(sp.latex(h_3dot))
h_3dot
# h_3dot_t1 = my_dhdx.T @ x_3dot + x_2dot.T @  (my_d2hdx2.T @ (A@xhat + B @ uhat))
# print(sp.latex(h_3dot_t1))

In [ ]:
d_h3dotdx = diff(h_3dot, x)
d_h3dotdx = d_h3dotdx.reshape(24, 1)
d_h3dotdx = Matrix(d_h3dotdx)
d_h3dotdx

In [ ]:
h_4dot = d_h3dotdx.T @ (A@xhat + B@uhat)
h_4dot.simplify()
h_4dot

In [ ]:
print(sp.latex(h_4dot))

In [ ]:
#for this model we need h_4dot because it's the only one that will actually have a component for pitch and roll to be able to control the system,